In [1]:
import pandas as pd
from tqdm import tqdm
from history_reader import *
from os import listdir

# 옵션 분석

In [2]:
history = pd.DataFrame([])
for filename in listdir('cube_history'):
    temp = reader(f'cube_history/{filename}')
    temp = option_preprocessor(temp)
    history = pd.concat([history, temp], axis=0)
del temp
history.sort_values(by='create_date', inplace=True)
history.reset_index(drop=True, inplace=True)

In [3]:
history.head()

,create_date,cube_type,item_equip_part,item_level,target_item,potential_option_grade,after_potential_option_1,after_potential_option_2,after_potential_option_3
0,2023-10-13 13:13:09+09:00,블랙 큐브,장갑,200,아케인셰이드 나이트글러브,레전드리,최대 HP : +12%,올스탯 : +6%,피격 시 5% 확률로 데미지의 20% 무시
1,2023-10-13 13:13:11+09:00,블랙 큐브,장갑,200,아케인셰이드 나이트글러브,레전드리,STR : +12%,피격 시 5% 확률로 데미지의 20% 무시,LUK : +9%
2,2023-10-13 13:13:13+09:00,블랙 큐브,장갑,200,아케인셰이드 나이트글러브,레전드리,최대 HP : +12%,올스탯 : +6%,피격 시 5% 확률로 데미지의 20% 무시
3,2023-10-13 13:13:15+09:00,블랙 큐브,장갑,200,아케인셰이드 나이트글러브,레전드리,최대 MP : +12%,피격 시 5% 확률로 데미지의 20% 무시,올스탯 : +6%
4,2023-10-13 13:13:17+09:00,블랙 큐브,장갑,200,아케인셰이드 나이트글러브,레전드리,최대 MP : +12%,올스탯 : +6%,최대 HP : +9%


In [4]:
options_list = [
    'STR : +12%',
    'DEX : +12%',
    'INT : +12%',
    'LUK : +12%',
    '최대 HP : +12%',
    '최대 MP : +12%',
    '크리티컬 데미지 : +8%',
    '올스탯 : +9%',
    '피격 시 10% 확률로 데미지의 20% 무시',
    '피격 시 10% 확률로 데미지의 40% 무시',
    '<쓸만한 윈드 부스터> 스킬 사용 가능'
]

In [5]:
def option_prop(option_name, option_num, history):
    count = (history[f'after_potential_option_{option_num}']==option_name).sum()
    prop = count / history.shape[0]
    print(f'옵션 [{option_name}]이(가) {option_num}번째 줄에서 등장한 비율은 {prop:5.1%}({count:3d}회)입니다.')
    return prop

In [6]:
prop = option_prop('크리티컬 데미지 : +8%', 1, history)
prop = option_prop('크리티컬 데미지 : +8%', 2, history)
prop = option_prop('크리티컬 데미지 : +8%', 3, history)

옵션 [크리티컬 데미지 : +8%]이(가) 1번째 줄에서 등장한 비율은  9.6%( 67회)입니다.
옵션 [크리티컬 데미지 : +8%]이(가) 2번째 줄에서 등장한 비율은  2.1%( 15회)입니다.
옵션 [크리티컬 데미지 : +8%]이(가) 3번째 줄에서 등장한 비율은  0.3%(  2회)입니다.
